# Author:
## Zashank Khare

#### Creating Spark Session

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Linear_Regression').getOrCreate()

#### Importing important libraries

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

from pyspark.ml.linalg import Vectors     
# Vectors class is used to represent dense and sparse vectors in Spark.

from pyspark.ml.feature import VectorAssembler  
# It is used in Spark ML pipelines, where it helps prepare data for ML algorithms that expect a single vector column as input.

from pyspark.ml.regression import LinearRegression

#### Loading datasets

In [3]:
df = spark.read.csv('Ecommerce_Customers.csv', header = True, inferSchema = True)

#### First five rows of df

In [4]:
df.show(5)

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [5]:
for x in df.head(1)[0]:
    print(x)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


#### Schema of df

In [6]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
print('No. of records present in the df: ',df.count())

No. of records present in the df:  500


#### Descriptive Statistics

In [8]:
from pyspark.sql.functions import round, col
stats = df.summary()
stats.select(stats['summary'],*[round(stats[c],2).alias(c) for c in stats.columns if c != 'summary']).show()

+-------+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|summary|Email|Address|Avatar|Avg Session Length|Time on App|Time on Website|Length of Membership|Yearly Amount Spent|
+-------+-----+-------+------+------------------+-----------+---------------+--------------------+-------------------+
|  count|500.0|  500.0| 500.0|             500.0|      500.0|          500.0|               500.0|              500.0|
|   mean| null|   null|  null|             33.05|      12.05|          37.06|                3.53|             499.31|
| stddev| null|   null|  null|              0.99|       0.99|           1.01|                 1.0|              79.31|
|    min| null|   null|  null|             29.53|       8.51|          33.91|                0.27|             256.67|
|    25%| null|   null|  null|             32.34|      11.39|          36.34|                2.93|             444.97|
|    50%| null|   null|  null|             33.08

In [9]:
The data looks good. 

SyntaxError: invalid syntax (2333874015.py, line 1)

In [ ]:
df.distinct().count()

In [ ]:
df.select('Avatar').distinct().count()

In [ ]:
print('Showing the average of all the numerical features per Avatar:')
for c in ['Avg Session Length','Time on App','Time on Website','Length of Membership','Yearly Amount Spent']:
    var = ''
    var = 'avg'+'('+ c + ')'
    view1 = df.select('Avatar',col(c)).groupBy('Avatar').mean()
    view1 = view1.select('Avatar',round(col(var),2).alias(c)).orderBy(c, ascending = False)
    print(view1.show(5))

#### Model and Fit

In [ ]:
df.columns

In [ ]:
Cols = ['Avg Session Length','Time on App','Time on Website','Length of Membership']

In [ ]:
assembler = VectorAssembler(inputCols = Cols, outputCol='features')
assem_data = assembler.transform(df)

In [ ]:
assem_data.head()

In [ ]:
final_data = assem_data.select('features','Yearly Amount Spent')

In [ ]:
final_data.show()

In [ ]:
train, validation = final_data.randomSplit([.8,.2])

In [ ]:
train.summary().show()

In [ ]:
validation.summary().show()

In [ ]:
Regressor = LinearRegression(labelCol= 'Yearly Amount Spent')
model = Regressor.fit(train)

In [ ]:
validation_result = model.evaluate(validation)

In [ ]:
validation_result.residuals.show()

In [ ]:
validation_result.rootMeanSquaredError

In [ ]:
# Models performs good as the RMSE value is acceptable as comparison to the mean, std of the target variable.

In [ ]:
df.select('Yearly Amount Spent').summary().show()

In [ ]:
validation_result.meanAbsoluteError

In [ ]:
validation_result.meanSquaredError

In [ ]:
validation_result.r2

In [ ]:
# R2 value is also good as it saying the model explains the 98.3% of the variances of the data

## Thank You
#### Happy Learning